In [1]:
import pixiedust
from pyspark.sql import SparkSession

pixiedust.installPackage("org.apache.bahir:spark-sql-cloudant_2.11:0")
spark = SparkSession.builder.getOrCreate()

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20191223150628-0001
KERNEL_ID = 3762eff8-ac0d-4656-a052-4eb69aaf1279
Pixiedust database opened successfully


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Package org.apache.bahir:spark-sql-cloudant_2.11:2.4.0 downloaded successfully
Please restart Kernel to complete installation of the new package
Successfully added package org.apache.bahir:spark-sql-cloudant_2.11:2.4.0


In [2]:
#!pip install --user --upgrade pixiedust

In [3]:
def readDataFrameFromCloudant(host,user,pw,database):

    cloudantdata=spark.read.format("org.apache.bahir.cloudant"). \
    option("cloudant.host",host). \
    option("cloudant.username", user). \
    option("cloudant.password", pw). \
    load(database)

    cloudantdata.createOrReplaceTempView("washing")
    #spark.sql("SELECT * from washing").show()
    return cloudantdata

In [4]:
#TODO Please provide your Cloudant credentials here
hostname = ""
user = ""
pw = ""
database = "washing" #as long as you didn't change this in the NodeRED flow the database name stays the same
cloudantdata=readDataFrameFromCloudant(hostname, user, pw, database)

In [6]:
# You should see some data coming from the database
spark.sql("select * from washing").show()
# spark.sql("select avg(voltage), std(voltage), max(voltage) from washing").show()

+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|                 _id|                _rev|count|flowrate|fluidlevel|frequency|hardness|speed|temperature|           ts|voltage|
+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|19cb0e12be2e77bcd...|1-b03781e80c30e6a...|    1|    null|      null|       77|    null| null|       null|1577112519529|    229|
|19cb0e12be2e77bcd...|1-ae84016fd49cd0b...|   14|      11|acceptable|     null|      70| null|         82|1577112530595|   null|
|19cb0e12be2e77bcd...|1-cde95532a0ebd26...|   18|      11|acceptable|     null|      77| null|         94|1577112534612|   null|
|19cb0e12be2e77bcd...|1-2ca2f9b93ff8a46...|   21|      11|acceptable|     null|      75| null|         88|1577112537631|   null|
|19cb0e12be2e77bcd...|1-8559ea7c1f87a14...|    8|    null|      null|       64|    null| null|   

In [7]:
df = spark.sql("""
select mean(voltage) as mean, std(voltage) as std from (
    SELECT ROW_NUMBER() OVER (ORDER BY ts desc) AS row, * FROM washing where voltage is not null
) where
row < 30
""")
               
#df.show()
mean = df.first().mean
std = df.first().std
#spark.sql("SELECT * from (select abs( ({0}-voltage)/({1}+0.0001) ) as zscore, voltage,ROW_NUMBER() OVER (ORDER BY ts desc) AS row FROM washing where voltage is not null order by ts desc) where row < 20".format(mean,std)).show()
df = spark.sql("SELECT mean(zscore) as avgzscore from (select abs( ({0}-voltage)/({1}+0.0001) ) as zscore, voltage,ROW_NUMBER() OVER (ORDER BY ts desc) AS row FROM washing where voltage is not null order by ts desc) where row < 20".format(mean,std))
df.show()

+------------+
|   avgzscore|
+------------+
|0.9880080000|
+------------+



In [8]:
zscore = df.first().avgzscore

In [9]:
import requests
if (zscore > 0.5):
    r = requests.get("https://1234thisismyname.mybluemix.net/edgemodelupdate?command=emergencyshutdown")
    r.status_code